<a href="https://colab.research.google.com/github/ryu622/gnn-counterattack-xai-v2/blob/fix%2Ffile-clean/GAT_CounterAttack_Prediction_Train_Scientific7_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#シード値
import random
import os
import numpy as np
import torch

def set_seed(seed=42):
    # Python自体の乱数固定
    random.seed(seed)
    # OS環境の乱数固定
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Numpyの乱数固定
    np.random.seed(seed)
    # PyTorchの乱数固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # マルチGPUの場合
    # 計算の決定論的挙動を強制（これを入れると少し遅くなることがありますが、再現性は完璧になります）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 好きな数字（42が一般的）で固定
set_seed(42)

In [ ]:
#GoogleDriveをマウント

from google.colab import drive

# Google Driveを仮想ファイルシステムにマウント
drive.mount('/content/drive')

In [ ]:
# 必須モジュールのインポート
!pip install torch_geometric

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import optim

import pandas as pd
import os
from glob import glob
import numpy as np
import re
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# 表示設定
np.set_printoptions(suppress=True, precision=3)
pd.set_option('display.precision', 3)    # 小数点以下の表示桁
pd.set_option('display.max_rows', 50)   # 表示する行数の上限
pd.set_option('display.max_columns', 15)  # 表示する列数の上限
%precision 3

使用データは以前までと同様

In [ ]:
import torch
from torch_geometric.loader import DataLoader

# ==========================================
# 【v7専用】ロード・クリーンアップ・最終確認
# ==========================================
# さきほど保存した v7 のパスに合わせます
v7_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v14_final.pt"

try:
    print(f"v7 最終データをロード中: {v7_load_path}")
    # 統合済みファイルをロード
    checkpoint = torch.load(v7_load_path, weights_only=False)

    # v7 の Builder ですでに 7次元特徴量 (x, y, vx, vy, dist_goal, dist_ball, team_id)
    # を付与しているため、基本的にはそのまま DataLoader に渡せます。
    train_set = checkpoint['train_data']
    test_set = checkpoint['test_data']

    # DataLoader を構築 (バッチサイズはメモリに合わせて調整してください)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    print(f"--- ロード完了 ---")
    print(f"訓練セット: {len(train_set)} 枚")
    print(f"テストセット: {len(test_set)} 枚")

    # 【最重要チェック】ノード数と次元数の確認
    sample_train = train_set[0]
    sample_test = test_set[0]

    # 期待値: [23, 7] (22人 + ボール1つ、7種類の特徴量)
    print(f"訓練データの形状: {sample_train.x.shape}")
    print(f"テストデータの形状: {sample_test.x.shape}")

    # 1. 次元数チェック
    if sample_train.x.shape[1] == 7:
        print("次元数: OK (7次元)")
    else:
        print(f"次元数警告: {sample_train.x.shape[1]}次元になっています。")

    # 2. ノード数チェック
    if sample_train.x.shape[0] == 23:
        print("ノード数: OK (23ノード固定)")
    else:
        print(f"ノード数警告: {sample_train.x.shape[0]}ノードになっています。")

    # 3. 速度データの存在チェック
    # 2列目(vx)の絶対値平均が0でなければ、速度が正しく入っています
    v_mean = torch.abs(sample_train.x[:, 2]).mean().item()
    if v_mean > 0.01:
        print(f"物理量チェック: OK (平均速度属性を確認)")
    else:
        print("物理量警告: 速度が0に張り付いています。Builderを再確認してください。")

    if sample_train.x.shape[1] == 7 and v_mean > 0.01:
        print("\nすべての準備が整いました。PIGNN 学習を開始してください。")

except FileNotFoundError:
    print(f"ファイルが見つかりません。パスを確認してください: {v7_load_path}")
except Exception as e:
    print(f"エラー発生: {e}")

モデルの定義（PIGNNのオリジナルクラス）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
import matplotlib.pyplot as plt
import numpy as np

# ==========================================
# 1. モデル定義 (PIGNN: 改良版 7次元入力)
# ==========================================
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index, pos, vel):
        h = self.lin(x)
        return self.propagate(edge_index, x=h, pos=pos, vel=vel)

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i):
        # r(t + tau) = r(t) + v(t)tau
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau

        # 将来の近接度に基づく物理バイアス
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 5.0)

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        alpha = F.leaky_relu(alpha) + physics_bias

        alpha = softmax(alpha, edge_index_i)
        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        # 修正: 7次元 [x, y, vx, vy, d_goal, d_ball, team_id]
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = self.conv2(x, edge_index, pos, vel)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

# ==========================================
# 2. 前処理・損失関数の修正
# ==========================================
def preprocess_batch(data, device):
    # vx, vy の正規化
    #data.x[:, 2:4] = data.x[:, 2:4] / 10.0
    data.x[:, 2:4] = data.x[:, 2:4]
    # d_goal, d_ball の正規化
    data.x[:, 4:6] = data.x[:, 4:6] / 100.0

    # 【解説】data.x[:, 6] (team_id) は 0, 1, 2 なので正規化せずそのまま使う

    data.x = data.x.float()
    data.pos = data.pos.float()
    data.vel = data.vel.float()
    return data.to(device)

def pignn_loss_function(out, target_y, vel, alpha_p=0.01):
    classification_loss = F.nll_loss(out, target_y)
    # 運動学的ペナルティ: 12m/s超えへの制約
    v_magnitude = torch.norm(vel, dim=-1)
    l_motion = torch.mean(F.relu(v_magnitude - 12.0)**2)
    return classification_loss, l_motion

def train_pignn_epoch(model, loader, optimizer, alpha_p, device):
    model.train()
    total_loss, total_phys = 0, 0
    for data in loader:
        # データの次元数やデバイス転送を行う前処理
        data = preprocess_batch(data, device)

        optimizer.zero_grad()
        out = model(data)

        # 損失計算
        c_loss, p_loss = pignn_loss_function(out, data.y.view(-1), data.vel, alpha_p)
        loss = c_loss + alpha_p * p_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        total_phys += p_loss.item() * data.num_graphs

    return total_loss / len(loader.dataset), total_phys / len(loader.dataset)

def test_pignn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

In [ ]:
# ==========================================
# 4. 実行セクション (7次元入力・チーム情報考慮版)
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ハイパーパラメータ（これまでの設定を維持）
EPOCHS = 50
LR = 0.0005
ALPHA_P = 0.01

history = {
    'total_loss': [],
    'physics_loss': [],
    'test_acc': []
}

# 【重要】in_channels=7 を受け取るように定義したモデルを呼び出し
model = PIGNNClassifier(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print(f"PIGNN学習開始 (Device: {device})")
print("Input Features: 7 [x, y, vx, vy, d_goal, d_ball, team_id]")

for epoch in range(1, EPOCHS + 1):
    # 学習の実行
    avg_loss, avg_phys = train_pignn_epoch(model, train_loader, optimizer, ALPHA_P, device)

    # テスト評価
    acc = test_pignn(model, test_loader, device)

    # 履歴の保存
    history['total_loss'].append(avg_loss)
    history['physics_loss'].append(avg_phys)
    history['test_acc'].append(acc)

    # 進捗表示
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {avg_loss:.4f} | Phys_L: {avg_phys:.4f} | Acc: {acc:.4f}")

    # モデルの保存（ファイル名を v7 に更新しておく）
    if epoch == 1 or acc > max(history['test_acc'][:-1]):
        torch.save(model.state_dict(), 'best_pignn_model_v14.pth')
        print(f" >> Model Saved (Best Acc: {acc:.4f})")

print("\n学習が正常に完了しました。")

In [ ]:
# ==========================================
# 5. 最終評価とレポート (PIGNN : チーム情報考慮版)
# ==========================================
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 1. 保存したベストモデル（）をロード
# モデルのインスタンス(in_channels=7)は直前のセクションで作られたものを使います
model.load_state_dict(torch.load('best_pignn_model_v14.pth'))
model.eval()

all_preds = []
all_labels = []

# 2. テストデータで最終予測
with torch.no_grad():
    for data in test_loader:
        # preprocess_batch内でteam_id(data.x[:, 6])が正しく扱われます
        data = preprocess_batch(data, device)
        out = model(data)
        pred = out.argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.view(-1).cpu().numpy())

# 3. レポート表示
print("\n" + "="*60)
print("       PIGNN 最終評価結果 (物理情報+チーム属性考慮・v7版)")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=['Failure (0)', 'Success (1)']))

# 4. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (PIGNN v7 Team-Aware)')
plt.ylabel('Actual (True)')
plt.xlabel('Predicted')
plt.show()

以上がPIGNNのベースラインモデル

PFI

In [ ]:
import torch

# 1. 保存した最新の PIGNN モデルパスを指定
save_path = "best_pignn_model_v7.pth"

# 2. モデルのインスタンス化 (学習時と同じ PIGNNClassifier を使う)
# ※ PIGNNClassifier の定義が同じセルか以前のセルにある必要があります
model = PIGNNClassifier(hidden_channels=64)

# 3. 重みのロード
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if os.path.exists(save_path):
    # weights_only=True はセキュリティ上推奨される設定です
    checkpoint = torch.load(save_path, map_location=device, weights_only=True)
    model.load_state_dict(checkpoint)

    # 4. デバイス転送と評価モード
    model.to(device)
    model.eval()
    print(f"PIGNNモデル (v6) のロードに成功しました！精度 89% の重みが適用されています。")
else:
    print(f"エラー: {save_path} が見つかりません。保存したファイル名を確認してください。")

In [ ]:
# 最初のバッチを流してエラーが出ないかテスト
sample_data = next(iter(test_loader))

# 1. まずモデルをデバイスに送る（再確認）
model.to(device)
model.eval()

# 2. データをデバイスに送る (preprocess_batch 内で .to(device) されているか確認)
sample_data = preprocess_batch(sample_data, device)

with torch.no_grad():
    # 3. 推論を実行
    out = model(sample_data)
    print("推論テスト成功！ 出力形状:", out.shape) # [16, 2]

In [ ]:
def calculate_feature_importance_pignn(model, loader, device):
    model.eval()
    data_list = list(loader)

    # 特徴量リスト (6次元)
    node_features = ['pos_x', 'pos_y', 'vel_x', 'vel_y', 'dist_goal', 'dist_ball']
    all_names = node_features
    importances = []

    # ベースライン精度計測
    all_labels, all_preds = [], []
    with torch.no_grad():
        for data in data_list:
            data = preprocess_batch(data.clone())
            out = model(data)
            all_preds.extend(out.argmax(dim=1).cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())
    baseline_acc = (np.array(all_preds) == np.array(all_labels)).mean()

    for i in range(len(all_names)):
        preds = []
        with torch.no_grad():
            for data in data_list:
                batch_data = data.clone()
                batch_data = preprocess_batch(batch_data)
                # 該当特徴量を破壊
                batch_data.x[:, i] = 0
                out = model(batch_data)
                preds.extend(out.argmax(dim=1).cpu().numpy())

        acc = (np.array(preds) == np.array(all_labels)).mean()
        importances.append(max(0, baseline_acc - acc))

    return all_names, importances

In [ ]:
def calculate_feature_importance_pignn(model, loader, device):
    model.eval()
    data_list = list(loader)

    # 【重要修正】 特徴量リストを7次元に更新
    node_features = [
        'pos_x', 'pos_y',
        'vel_x', 'vel_y',
        'dist_goal', 'dist_ball',
        'team_id' # 7番目の新メンバー
    ]
    all_names = node_features
    importances = []

    # 1. ベースライン精度の計算
    all_labels, all_preds = [], []
    with torch.no_grad():
        for data in data_list:
            batch_data = preprocess_batch(data.clone(), device)
            out = model(batch_data)
            all_preds.extend(out.argmax(dim=1).cpu().numpy())
            all_labels.extend(batch_data.y.view(-1).cpu().numpy())

    baseline_acc = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"ベースライン精度 (Team-Aware v7): {baseline_acc:.4f}")

    # 2. 各特徴量を順番に破壊
    for i in range(len(all_names)):
        preds = []
        with torch.no_grad():
            for data in data_list:
                batch_data = data.clone()
                batch_data = preprocess_batch(batch_data, device)

                # i番目の特徴量を破壊
                batch_data.x[:, i] = 0

                out = model(batch_data)
                preds.extend(out.argmax(dim=1).cpu().numpy())

        acc = (np.array(preds) == np.array(all_labels)).mean()
        drop = baseline_acc - acc
        importances.append(max(0, drop))
        print(f"特徴量 '{all_names[i]}' 破壊時の精度低下: {drop:.4f}")

    return all_names, importances

# --- 実行 ---
feature_names, importance_values = calculate_feature_importance_pignn(model, test_loader, device)

アテンション係数の可視化

In [ ]:
#アテンション係数の可視化
import matplotlib.patches as patches

def visualize_pignn_final_v3(model, loader, sample_idx=0):
    model.eval()
    data = next(iter(loader)).to(device)

    # 推論
    input_data = data.clone()
    input_data = preprocess_batch(input_data, device)
    with torch.no_grad():
        out = model(input_data)
        probs = torch.exp(out)
        preds = out.argmax(dim=1)

    mask = (data.batch == sample_idx)
    pos = data.pos[mask].cpu().numpy()
    vel = data.vel[mask].cpu().numpy()

    # ノード数を確認（36個想定）
    num_nodes = pos.shape[0]

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('#f0f0f0')

    # ピッチ描画
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='black', lw=2))
    ax.plot([0, 0], [-34, 34], color='black', alpha=0.3)

    for i in range(num_nodes):
        # --- 判定ロジックを「インデックス」に変更 ---
        if i == num_nodes - 1: # 最後のノードがボール
            color, marker, size, z = 'gold', '*', 450, 15
        elif i < 11: # 最初の11人が味方（仮定）
            color, marker, size, z = 'blue', 'o', 180, 10
        elif i < 22: # 次の11人が敵（仮定）
            color, marker, size, z = 'red', 'o', 180, 10
        else: # それ以外のダミー等
            color, marker, size, z = 'gray', 'o', 100, 5

        # 描画
        ax.scatter(pos[i, 0], pos[i, 1], c=color, marker=marker, s=size, edgecolors='black', zorder=z)
        # ベクトル
        ax.arrow(pos[i, 0], pos[i, 1], vel[i, 0]*1.5, vel[i, 1]*1.5,
                 head_width=1.0, head_length=1.2, fc=color, ec=color, alpha=0.4, zorder=z-1)

    plt.title(f"PIGNN Fixed Visualization\nPred: {preds[sample_idx]} | Prob: {probs[sample_idx, 1]:.2%}")
    plt.show()

# 実行
visualize_pignn_final_v3(model, test_loader, sample_idx=0)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch

# 1. 関数の定義（引数に device を明示的に追加）
def visualize_pignn_v14_fixed(model, loader, sample_idx=0, device='cuda'):
    model.eval()

    # DataLoaderから1バッチ取得
    batch = next(iter(loader)).to(device)

    # 推論
    with torch.no_grad():
        out = model(batch)
        # Softmaxを適用して確率にする
        probs = torch.softmax(out, dim=1)
        preds = out.argmax(dim=1)

    # 指定したサンプル(sample_idx)を抽出
    mask = (batch.batch == sample_idx)
    node_features = batch.x[mask].cpu().numpy()

    # --- 座標の復元 (-1~1 -> 105x68m) ---
    pos_x = node_features[:, 0] * 52.5
    pos_y = node_features[:, 1] * 34
    # 速度ベクトル (3, 4列目)
    vel_x = node_features[:, 2] * 5
    vel_y = node_features[:, 3] * 5
    # チーム属性 (7列目)
    team_val = node_features[:, 6]

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('#2e7d32') # 芝生の色

    # ピッチ描画
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=True, color='#2e7d32', zorder=1))
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='white', lw=2, zorder=2))
    ax.plot([0, 0], [-34, 34], color='white', alpha=0.5, zorder=2) # センターライン

    for i in range(len(pos_x)):
        if team_val[i] == 2.0: # ボール
            color, marker, size, z = 'gold', '*', 400, 10
        elif team_val[i] == 0.0: # 攻撃側(Home)
            color, marker, size, z = 'blue', 'o', 150, 5
        else: # 守備側(Away)
            color, marker, size, z = 'red', 'o', 150, 5

        ax.scatter(pos_x[i], pos_y[i], c=color, marker=marker, s=size, edgecolors='white', zorder=z)
        # 速度の矢印
        ax.arrow(pos_x[i], pos_y[i], vel_x[i], vel_y[i],
                 head_width=1.0, head_length=1.2, fc='yellow', ec='yellow', alpha=0.6, zorder=z+1)

    actual_label = batch.y[sample_idx].item()
    plt.title(f"v14 Fixed Visualization\nActual: {actual_label} | Pred: {preds[sample_idx]} | Prob(Success): {probs[sample_idx, 1]:.2%}")
    plt.xlim(-60, 60)
    plt.ylim(-40, 40)
    plt.show()

# 2. 実行（device変数が定義されていない場合に備えて直接 'cuda' などを指定するか、定義を確認してください）
# もしエラーが出る場合は device='cpu' に変えてみてください
visualize_pignn_v14_fixed(model, test_loader, sample_idx=0, device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def visualize_attention_v14(model, loader, sample_idx=0, device='cuda'):
    model.eval()
    batch = next(iter(loader)).to(device)

    # 1. アテンション係数を取得しながら推論
    # ※モデル側に return_attention=True のような機能がある前提です
    with torch.no_grad():
        # モデルの仕様に合わせて調整が必要ですが、一般的にはこう取得します
        out, (edge_index, att_weights) = model.forward_with_attention(batch)
        probs = torch.softmax(out, dim=1)

    # 特定のサンプルのマスク
    mask = (batch.batch == sample_idx)
    node_features = batch.x[mask].cpu().numpy()

    # 座標復元
    pos_x = node_features[:, 0] * 52.5
    pos_y = node_features[:, 1] * 34
    team_val = node_features[:, 6]

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('#1a472a') # 濃い緑

    # 2. アテンション（エッジ）を描画
    # sample_idxに該当するエッジだけをループ
    # att_weights が高いものほど太く、明るい色で描く
    for i in range(edge_index.shape[1]):
        src, dst = edge_index[0, i], edge_index[1, i]
        # このサンプル内のノードかチェック（簡易版）
        if src < len(pos_x) and dst < len(pos_x):
            weight = att_weights[i].item()
            if weight > 0.1: # 重要な繋がりだけ描画
                ax.plot([pos_x[src], pos_x[dst]], [pos_y[src], pos_y[dst]],
                        color='white', alpha=weight, lw=weight*5, zorder=3)

    # 3. 選手を描画（先ほどと同じ）
    for i in range(len(pos_x)):
        color = 'gold' if team_val[i] == 2.0 else ('blue' if team_val[i] == 0.0 else 'red')
        ax.scatter(pos_x[i], pos_y[i], c=color, s=150, edgecolors='white', zorder=5)

    plt.title("PIGNN Attention Visualization\n(Lines show where AI is looking)")
    plt.show()
visualize_attention_v14(model, test_loader, sample_idx=0, device='cuda' if torch.cuda.is_available() else 'cpu')

学習曲線

In [ ]:
import matplotlib.pyplot as plt

# 学習時のループ内で train_acc も記録するように修正して実行したと仮定
# もし記録していなければ、このコードで現在の history からグラフを出します
plt.figure(figsize=(10, 6))
plt.plot(history['total_loss'], label='Train Loss (Total)')
# もし train_acc を取っていればここに追加
plt.plot(history['test_acc'], label='Test Accuracy', marker='o')

plt.title('Check for Overfitting: Loss vs Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

軽度の過学習だが、データサイズ的に仕方がないらしい。
訓練曲線が減少から一定に落ち着いているのに対して、テスト曲線が上がっていたら過学習の傾向。今回の場合はテスト曲線が乱降下しているので軽度？

検証

ラベルをシャッフルした時の精度

In [ ]:
def shuffle_label_test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            # ラベルをランダムにシャッフルする
            random_y = data.y[torch.randperm(data.y.size(0))]

            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == random_y.view(-1)).sum().item()
            total += data.num_graphs

    print(f"ラベルシャッフル時の精度: {correct / total:.4f}")
    print(">> 0.5 (50%) 前後になれば、モデルは正しくラベルと特徴の関係を学んでいます。")

shuffle_label_test(model, test_loader, device)

物理損失（損失関数に付け加えた物理項）

In [ ]:
# 物理損失の平均値を算出
avg_phys = sum(history['physics_loss']) / len(history['physics_loss'])
print(f"平均物理損失: {avg_phys:.4f}")

if avg_phys < 25: # 18前後なら非常に優秀
    print(">> 物理的整合性は保たれています。AIは現実的な動きの範囲内で予測しています。")
else:
    print(">> 物理損失が高いです。AIが異常な速度を想定して予測している可能性があります。")

過学習を防ぐためにドロップアウト層を追加した修正版PIGNNモデル

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

class PIGNNClassifier_drop(nn.Module):
    def __init__(self, hidden_channels=64, dropout_rate=0.3):
        super(PIGNNClassifier_drop, self).__init__()
        self.dropout_rate = dropout_rate

        # 修正: 7次元入力 [x, y, vx, vy, d_goal, d_ball, team_id]
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        # 1層目 + Dropout
        x = self.conv1(x, edge_index, pos, vel)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # 2層目 + Dropout
        x = self.conv2(x, edge_index, pos, vel)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = global_mean_pool(x, batch)

        # 最終出力
        return F.log_softmax(self.lin(x), dim=1)

class PIGNNClassifier_v7_Final(nn.Module):
    def __init__(self, hidden_channels=64, dropout_rate=0.3):
        super(PIGNNClassifier_v7_Final, self).__init__()
        self.dropout_rate = dropout_rate

        # 修正: 7次元 [x, y, vx, vy, d_goal, d_ball, team_id]
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        # 1層目 + Dropout
        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # 2層目 + Dropout
        x = F.elu(self.conv2(x, edge_index, pos, vel))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

In [ ]:
# 1. 新しいモデルをインスタンス化
model_dropout = PIGNNClassifier_drop(hidden_channels=64, dropout_rate=0.3).to(device)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=LR)

# 2. 履歴保存用（名前を分ける：history_dropout）
history_dropout = {
    'total_loss': [],
    'physics_loss': [],
    'test_acc': []
}

print(f"PIGNN学習開始（ドロップアウト版）")

for epoch in range(1, EPOCHS + 1):
    # model_dropout を使う
    avg_loss, avg_phys = train_pignn_epoch(model_dropout, train_loader, optimizer_dropout, ALPHA_P, device)
    acc = test_pignn(model_dropout, test_loader, device)

    history_dropout['total_loss'].append(avg_loss)
    history_dropout['physics_loss'].append(avg_phys)
    history_dropout['test_acc'].append(acc)

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {avg_loss:.4f} | Phys_L: {avg_phys:.4f} | Acc: {acc:.4f}")

    # 保存ファイル名を変える（重要！）
    if epoch == 1 or acc > max(history_dropout['test_acc'][:-1]):
        torch.save(model_dropout.state_dict(), 'best_pignn_model_v6_dropout.pth')
        print(f" >> Model Saved (Best Dropout Acc: {acc:.4f})")

精度曲線

In [ ]:
#ドロップアウト層なしとありを重ねて描画

plt.figure(figsize=(10, 6))
plt.plot(history['test_acc'], label='Base Model (No Dropout)', alpha=0.6)
plt.plot(history_dropout['test_acc'], label='Improved Model (With Dropout)', linewidth=2)
plt.title('Effect of Dropout on Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

ベースラインより乱高下の幅が小さくなっている→改善ポイント

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ==========================================
# 5. 最終評価とレポート (Dropout版・専用)
# ==========================================

# 1. ドロップアウト版のモデルをロード
# 学習時に保存した「dropout」という名前の方を読み込みます
model_drop_eval = PIGNNClassifier_drop(hidden_channels=64, dropout_rate=0.3).to(device)
model_drop_eval.load_state_dict(torch.load('best_pignn_model_v6_dropout.pth'))
model_drop_eval.eval()

all_preds = []
all_labels = []

# 2. テストデータで最終予測
with torch.no_grad():
    for data in test_loader:
        data = preprocess_batch(data, device)
        out = model_drop_eval(data)
        pred = out.argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.view(-1).cpu().numpy())

# 3. レポート表示
print("\n" + "="*50)
print("       PIGNN 最終評価結果 (Dropout改良版)")
print("="*50)
print(classification_report(all_labels, all_preds, target_names=['Failure (0)', 'Success (1)']))

# 4. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', # 区別するために色をオレンジ系に
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (PIGNN with Dropout)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

検証

In [ ]:
def shuffle_label_test_dropout_ver(model_to_test, loader, device):
    model_to_test.eval()
    correct = 0
    total = 0

    # 判定用の閾値（不均衡データの場合、多数派の割合に引っ張られるため）
    # 今回のテストデータ Failure:239, Success:90 なので、ランダムなら
    # (239/329)^2 + (90/329)^2 ≒ 0.6 くらいになるのが統計学的な「勘」の限界です。

    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)

            # ラベルをランダムにシャッフル
            random_y = data.y[torch.randperm(data.y.size(0))]

            out = model_to_test(data)
            pred = out.argmax(dim=1)

            correct += (pred == random_y.view(-1)).sum().item()
            total += data.num_graphs

    shuffle_acc = correct / total
    print(f"ラベルシャッフル時の精度: {shuffle_acc:.4f}")

    if shuffle_acc < 0.58: # 0.62から下がっていれば改善
        print(">> 合格：暗記（過学習）が抑制され、特徴量とラベルの真の相関を学んでいます。")
    else:
        print(">> 警告：依然としてデータの偏り（初期配置など）を強く覚えすぎている可能性があります。")

# 実行（ドロップアウト版のモデルを指定）
shuffle_label_test_dropout_ver(model_drop_eval, test_loader, device)

In [ ]:
# ==========================================
# 物理的整合性の最終確認 (Dropout版)
# ==========================================

# history_dropout の中身を使って計算します
if 'physics_loss' in history_dropout and len(history_dropout['physics_loss']) > 0:
    avg_phys = sum(history_dropout['physics_loss']) / len(history_dropout['physics_loss'])
    print(f"ドロップアウト版 平均物理損失: {avg_phys:.4f}")

    # 判定
    if avg_phys < 25:
        print(">> 合格：物理的整合性は保たれています。")
        print(">> ドロップアウトを導入しても、AIは現実的な物理法則（速度ベクトル）を無視していません。")
    else:
        print(">> 警告：物理損失が増大しています。")
        print(">> 汎化性能を優先するあまり、物理レイヤーの制約が弱まっている可能性があります。")
else:
    print(">> エラー：history_dropout に physics_loss が記録されていません。")

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.data import Data

# --- 修正の核心：重要度を抽出する1行 ---
# node_mask は (ノード数, 特徴量数) の形状なので、全ノードで平均して特徴量ごとの重要度にする
importances = explanation.node_mask.mean(dim=0)

# --- 以降、あなたの可視化コードへ続く ---
if torch.is_tensor(importances):
    importances = importances.cpu().numpy()

# 1. バラバラの引数を「dataオブジェクト」に梱包してモデルに渡すラッパー
class ExplainerWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x, edge_index, batch=None, **kwargs):
        # GNNExplainerから届く各テンソルを、モデルが期待する Data オブジェクトに擬似再現
        # pos と vel は x の中に入っている、あるいは別途渡されることを想定
        # あなたのモデル定義に合わせて x から pos, vel を切り出す、
        # または data.pos, data.vel としてアクセスできるようにします。

        # 仮に x の 0-1列目が pos, 2-3列目が vel だと想定される場合：
        # (モデルの入力に合わせて調整してください。x自体に全て含まれているならそのままでも可)
        tmp_data = Data(x=x, edge_index=edge_index, batch=batch)

        # もしモデルが data.pos や data.vel を直接参照しているなら、ここで代入
        # x の構成が [特徴量...] で、pos/velが別管理なら以下のように復元
        tmp_data.pos = x[:, :2]  # 例: 最初の2列が座標
        tmp_data.vel = x[:, 2:4] # 例: 次の2列が速度

        return self.model(tmp_data)

# ラップしたモデルを作成
wrapped_model = ExplainerWrapper(model_drop_eval)

# 2. Explainerの設定
model_config = {
    'mode': 'multiclass_classification',
    'task_level': 'graph',
    'return_type': 'log_probs',
}

explainer = Explainer(
    model=wrapped_model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=model_config,
)

# --- データの準備 ---
test_batch = next(iter(test_loader))
test_batch = preprocess_batch(test_batch, device)
data_list = test_batch.to_data_list()
data_single = data_list[0]

# 3. 重要度の算出
explanation = explainer(
    x=data_single.x,
    edge_index=data_single.edge_index,
    batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
)

print("重要度の算出に成功しました！")

# --- 修正版：重要度の可視化コード ---

# ラベルを7次元用に更新
labels = ['x', 'y', 'vx', 'vy', 'dist_goal', 'dist_ball', 'team_id']

# importances が numpy 形式であることを確認
if torch.is_tensor(importances):
    importances = importances.cpu().numpy()

plt.figure(figsize=(10, 6))
# ここで次元数を自動で合わせます
plt.bar(labels, importances, color='teal')

plt.title('GNNExplainer: Feature Importance (PIGNN v7)')
plt.ylabel('Importance Score')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 値を棒の上に表示
for i, v in enumerate(importances):
    plt.text(i, v + 0.01, f"{v:.3f}", ha='center')

plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. 解析の設定
num_samples = 100
all_node_importances = []

print(f"{num_samples}シーンの解析を開始します。勾配計算を有効にして最適化を行うため、少し時間がかかります...")

# モデルを評価モードにしつつ、GNNExplainer内部の学習は許可する
model_drop_eval.eval()

# 進捗管理用のカウンタ
count = 0

for data in tqdm(test_loader):
    if count >= num_samples:
        break

    data = preprocess_batch(data, device)
    data_list = data.to_data_list()

    for data_single in data_list:
        if count >= num_samples:
            break

        # --- 修正ポイント：with torch.no_grad() を削除 ---
        # GNNExplainerは内部でロスを計算し .backward() を呼ぶため勾配が必要
        explanation = explainer(
            x=data_single.x,
            edge_index=data_single.edge_index,
            batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
        )

        node_importance = explanation.node_mask.abs().mean(dim=0).cpu().numpy()
        all_node_importances.append(node_importance)
        count += 1

# 3. 平均と標準誤差の計算
avg_importance = np.mean(all_node_importances, axis=0)
std_importance = np.std(all_node_importances, axis=0) / np.sqrt(len(all_node_importances))

# --- 修正版：ラベルとグラフ描画 ---

# 1. ラベルを現在の7次元仕様に完全に合わせる
# [x, y, vx, vy, dist_goal, dist_ball, team_id]
labels = ['PosX', 'PosY', 'VelX', 'VelY', 'DistGoal', 'DistBall', 'TeamID']

plt.figure(figsize=(12, 7))

# 2. データの数とラベルの数が一致しているか確認して描画
# avg_importance と std_importance が 7要素であることを前提とします
bars = plt.bar(labels, avg_importance, yerr=std_importance,
               color='teal', edgecolor='navy', capsize=5, alpha=0.8)

plt.title('GNNExplainer: Mean Feature Importance over 100 Scenes (v7)', fontsize=14)
plt.ylabel('Importance Score', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# 棒の上に数値を表示
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 成功と失敗の比較解析 (PIGNN v7対応版)
# ==========================================
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. データの仕分け用リスト
success_importances = []
failure_importances = []

num_samples = 150  # 統計的安定性のために150シーンを推奨
count = 0

print(f"{num_samples}シーンを成功/失敗別に解析します...")

# モデルを評価モードに
model_drop_eval.eval()

# tqdmで進捗を表示しながらループ
for data in tqdm(test_loader):
    if count >= num_samples:
        break

    # バッチをデバイスに送り、個別データに分解
    data = preprocess_batch(data, device)
    data_list = data.to_data_list()

    for data_single in data_list:
        if count >= num_samples:
            break

        # GNNExplainerで重要度算出
        explanation = explainer(
            x=data_single.x,
            edge_index=data_single.edge_index,
            # batchテンソルもデバイスに合わせる
            batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
        )

        # ノード特徴量の重要度（絶対値の平均）を取得
        node_importance = explanation.node_mask.abs().mean(dim=0).cpu().numpy()

        # 正解ラベル(y)に基づいて仕分け
        if data_single.y.item() == 1:
            success_importances.append(node_importance)
        else:
            failure_importances.append(node_importance)

        count += 1

# 2. 【重要】ラベルを7次元（v7仕様）に更新
labels = ['PosX', 'PosY', 'VelX', 'VelY', 'DistGoal', 'DistBall', 'TeamID']

# 各グループの平均を算出
# ここで avg_success の shape は (7,) になります
avg_success = np.mean(success_importances, axis=0)
avg_failure = np.mean(failure_importances, axis=0)

# 3. 比較グラフの描画
x = np.arange(len(labels)) # 0から6までのインデックス
width = 0.35

fig, ax = plt.subplots(figsize=(14, 8))

# 棒グラフの描画
rects1 = ax.bar(x - width/2, avg_success, width, label='Success (1)', color='forestgreen', alpha=0.8)
rects2 = ax.bar(x + width/2, avg_failure, width, label='Failure (0)', color='crimson', alpha=0.8)

# グラフの装飾
ax.set_ylabel('Mean Importance Score', fontsize=12)
ax.set_title('Feature Importance Comparison: Success vs Failure (PIGNN v7)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.3)

# 数値ラベルを表示する補助関数
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), # 3pt上に表示
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

# 4. 数値の要約表示
print("\n--- 解析結果の要約 ---")
for i, label in enumerate(labels):
    diff = avg_success[i] - avg_failure[i]
    trend = "↑ Successで重視" if diff > 0 else "↓ Failureで重視"
    print(f"[{label}] Success: {avg_success[i]:.4f} | Failure: {avg_failure[i]:.4f} | {trend}")

In [ ]:
# 描画の前にこれを入れてください
sample = next(iter(test_loader))
# 最初の1バッチ分のチームIDの中身をすべて表示
print("--- TeamID Raw Data Check ---")
print(sample.x[:, 6])
print("-----------------------------")

# もしここで 0.0 しか出てこないなら、データの作り直しが必要。

In [ ]:
def analyze_physics_bias(model, data, tau=1.5):
    """
    モデル内の物理バイアスを解析する関数（名前を修正）
    """
    model.eval()
    with torch.no_grad():
        pos = data.pos
        vel = data.vel
        edge_index = data.edge_index

        # 未来位置の予測
        pos_pred = pos + vel * tau

        # エッジごとの未来距離と物理バイアス
        row, col = edge_index
        dist_future = torch.norm(pos_pred[row] - pos_pred[col], dim=-1)
        physics_bias = torch.exp(-dist_future / 5.0)

        # 最大バイアスの取得
        top_idx = torch.argmax(physics_bias)
        top_pair = (edge_index[0, top_idx].item(), edge_index[1, top_idx].item())

    # 戻り値のキーを 'max_bias' に修正してエラーを解消
    return {
        "top_pair": top_pair,
        "max_bias": physics_bias[top_idx].item(), # ここを bias_value から max_bias へ修正
        "pos_pred": pos_pred
    }

In [ ]:
def visualize_pignn_v7_absolute_colors(model, loader, sample_idx=0, tau=1.5):
    model.eval()
    # データを1つ取得
    batch = next(iter(loader)).to(device)
    input_data = preprocess_batch(batch.clone(), device)

    with torch.no_grad():
        out = model(input_data)
        probs = torch.exp(out)
        preds = out.argmax(dim=1)

    # 描画対象のインデックスを抽出
    mask = (batch.batch == sample_idx)
    pos = batch.pos[mask].cpu().numpy()
    vel = batch.vel[mask].cpu().numpy()

    # 【最重要】team_id (index 6) を直接取得して中身を確認
    team_ids = input_data.x[mask, 6].cpu().numpy()

    fig, ax = plt.subplots(figsize=(12, 8))
    # ピッチ背景
    ax.set_facecolor('#f0f0f0')
    ax.add_patch(plt.Rectangle((-52.5, -34), 105, 68, fill=False, color='black', lw=2))
    ax.plot([0, 0], [-34, 34], color='black', alpha=0.3)

    for i in range(len(pos)):
        t_val = team_ids[i]

        # --- 判定ロジックを「範囲」にして誤差を許容 ---
        if t_val > 1.5:          # Ball (2.0)
            color, marker, size, z = '#FFD700', '*', 500, 15 # Gold
            lbl = "Ball"
        elif t_val > 0.5:        # Defender (1.0)
            color, marker, size, z = '#EE3333', 'o', 250, 10 # Red
            lbl = "Defender (Away)"
        else:                    # Attacker (0.0)
            color, marker, size, z = '#3366FF', 'o', 250, 10 # Blue
            lbl = "Attacker (Home)"

        # 描画
        ax.scatter(pos[i, 0], pos[i, 1], c=color, marker=marker, s=size,
                   edgecolors='black', linewidths=1.2, zorder=z, label=lbl)

        # 速度ベクトル
        ax.arrow(pos[i, 0], pos[i, 1], vel[i, 0]*tau, vel[i, 1]*tau,
                 head_width=0.8, head_length=1.0, fc=color, ec=color,
                 alpha=0.3, zorder=z-1)

    # 物理バイアスの描画（緑のX印と点線）
    res = analyze_physics_bias(model, batch.to_data_list()[sample_idx], tau=tau)
    p1, p2 = res["top_pair"] # タプルなのでそのまま受け取るだけでOK
    p1, p2 = int(p1), int(p2) # 念のため整数型に変換
    ax.plot([pos[p1,0], pos[p2,0]], [pos[p1,1], pos[p2,1]], 'green', linestyle='--', lw=2, alpha=0.6)
    ax.scatter(res["pos_pred"][p1,0].cpu(), res["pos_pred"][p1,1].cpu(),
               color='green', marker='X', s=350, edgecolors='white', label='Conflict Point', zorder=20)

    # テキスト情報
    res_str = "SUCCESS" if preds[sample_idx] == 1 else "FAILURE"
    plt.title(f"PIGNN v7 Tactical Analysis: {res_str}\n"
              f"AI Prediction Prob: {probs[sample_idx, 1]:.2%} | Max Physics Bias: {res['max_bias']:.3f}",
              fontsize=14, fontweight='bold')

    # 凡例の重複を削除
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right', frameon=True, shadow=True)

    plt.tight_layout()
    plt.show()

# 実行
visualize_pignn_v7_absolute_colors(model, test_loader, sample_idx=0)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader

# ==========================================
# 1. テストセット全体から最小バイアスを探索
# ==========================================
model.eval()
min_bias = float('inf')
low_bias_data = None
low_bias_idx = -1

print("テストセットから最も物理的に安定したシーンを探索中...")

with torch.no_grad():
    # test_set は Data オブジェクトのリストであることを前提としています
    for i, data in enumerate(test_set):
        # データのデバイス移動
        data_to_device = data.to(device)

        # 解析関数を呼び出し（tau=1.5秒後の未来を予測）
        res = analyze_physics_bias(model, data_to_device, tau=1.5)

        current_max_bias = res['max_bias']

        # 最小値を更新
        if current_max_bias < min_bias:
            min_bias = current_max_bias
            low_bias_idx = i
            low_bias_data = data_to_device

print(f"探索完了")
print(f"発見された最小バイアス: {min_bias:.4f}")
print(f"該当データのインデックス: {low_bias_idx}")

# ==========================================
# 2. 修正版：IndexError回避用可視化呼び出し
# ==========================================

def visualize_specific_scene(model, data_list, target_idx, tau=1.5):
    """
    特定のインデックスのデータだけを抽出し、
    バッチとして可視化関数に渡すことで IndexError を防ぐ
    """
    # ターゲットのデータ1枚だけを含むリストを作成
    single_data_list = [data_list[target_idx]]

    # バッチサイズ1の専用ローダーを作成
    single_loader = DataLoader(single_data_list, batch_size=1)

    # 既存の可視化関数を呼び出し
    # バッチ内のインデックスは必ず 0 になる
    visualize_pignn_v7_absolute_colors(model, single_loader, sample_idx=0, tau=tau)

# 実行：物理的に最も「綺麗」なシーンを描画
visualize_specific_scene(model, test_set, low_bias_idx, tau=1.5)

In [ ]:
def find_most_intense_duel(model, data_list, tau=1.5):
    model.eval()
    max_duel_bias = -1.0
    best_idx = -1

    print("攻守が最も激しく『ぶつかる』シーンを探索中...")

    with torch.no_grad():
        for i, data in enumerate(data_list):
            data_to_device = data.to(device)
            # 全ペアのバイアス詳細を取得
            res = analyze_physics_bias(model, data_to_device, tau=tau)

            p1, p2 = res["top_pair"]
            # チームIDを取得 (index 6)
            t1 = data.x[p1, 6].item()
            t2 = data.x[p2, 6].item()

            # 異なるチーム同士（0.0:Home vs 1.0:Away）の衝突のみをターゲットにする
            # 1.0 - 0.0 = 1.0 の絶対値で判定
            if abs(t1 - t2) == 1.0:
                if res['max_bias'] > max_duel_bias:
                    max_duel_bias = res['max_bias']
                    best_idx = i

    print(f"発見！ 最大攻守衝突バイアス: {max_duel_bias:.4f} (Index: {best_idx})")
    return best_idx

# 1. 激しい競り合いシーンを特定
duel_idx = find_most_intense_duel(model, test_set)

# 2. 可視化
if duel_idx != -1:
    visualize_specific_scene(model, test_set, duel_idx, tau=1.5)
else:
    print("条件に合うシーンが見つかりませんでした。")

In [ ]:
import torch

# 統合後のファイルをチェック
checkpoint = torch.load("/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v14_final.pt", weights_only=False)
train_set = checkpoint['train_data']

# 全フレームの平均速度(vx)をリスト化
all_v_means = [torch.abs(d.x[:, 2]).mean().item() for d in train_set]

print(f"全{len(train_set)}フレーム中、速度が0のフレーム数: {all_v_means.count(0.0)}")
print(f"最初の5枚の速度平均: {all_v_means[:5]}")